In [1]:
import time
import math
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from requests.adapters import Retry
from requests.exceptions import Timeout
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException


In [2]:
firefox_options = Options()
firefox_options.headless = False


/tmp/ipykernel_5743/3424232155.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  firefox_options.headless = False


In [3]:
driver = webdriver.Firefox(options=firefox_options)

In [4]:
def scrape_page(url):
    driver.get(url)
    selenium_response = driver.page_source
    response = BeautifulSoup(selenium_response,"html.parser")
    return response

In [4]:
def scrape_page(url, max_retries=3):
    for attempt in range(max_retries):
        try:
            time.sleep(2)
            driver.get(url)
            selenium_response = driver.page_source
            soup = BeautifulSoup(selenium_response, "html.parser")
            return soup
        except Exception as e:
            attempt = attempt + 1
            print("Exception occurred on attempt", attempt + 1)
            print("Exception: ",e)
            if attempt + 1 < max_retries:
                print("Retrying...")
                time.sleep(2)  # Add a delay before retrying
                continue
            else:
                print("Max retries reached")
                return None


In [ ]:
def scrape_page(url):
    retry_strategy = Retry(
        total=3,
        status_forcelist=[429, 500, 502, 503, 504],
        backoff_factor=1
    )
    adapter = requests.adapters.HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    try:
        # response = session.get(url, timeout=5)
        driver.get(url)
        selenium_response = driver.page_source
    except Timeout:
        print("Timeout occurred")
        return None

    soup = BeautifulSoup(selenium_response, "html.parser")
    return soup

In [5]:
def get_cursor(soup, next_page_num):
    cursor = soup.find('a', {'class':'chakra-link css-1wxsdwr', 'aria-label': f'page {next_page_num}'})
    if cursor:
        return cursor.get('href')
    else:
        return None

In [6]:
def get_job_links(url, list_elems):
    job_links = []
    for item in list_elems:
        job_key = item.div.get('data-jobkey')
        job_link = f'{url}&job={job_key}'
        job_links.append(job_link)
    return job_links


In [7]:
def scrape_one_page(url, soup,search_keyword_for_job):
    ul = soup.find_all('ul', id = 'job-list')
    li = ul[0].find_all('li', {'class':'css-0'})
    job_links = get_job_links(url, li)
    data_list = []
    for job_link in job_links:
        print(job_link)
        job = scrape_page(job_link)
        job_title = job.h2.text.strip() if job.h2.text.strip() else ""
        company = job.find('span', {'data-testid': 'viewJobCompanyName'}).text if job.find('span', {'data-testid': 'viewJobCompanyName'}) else ""
        location = job.find('span', {'data-testid': 'viewJobCompanyLocation'}).text if job.find('span', {'data-testid': 'viewJobCompanyLocation'}) else ""
        job_type = job.find('span', {'data-testid': 'viewJobBodyJobDetailsJobType'}).text if job.find('span', {'data-testid': 'viewJobBodyJobDetailsJobType'}) else ""
        salary = job.find('span', {'data-testid': 'viewJobBodyJobCompensation'}).text if job.find('span', {'data-testid': 'viewJobBodyJobCompensation'}) else ""
        posted_on = job.find('span', {'data-testid': 'viewJobBodyJobPostingTimestamp'}).text if job.find('span', {'data-testid': 'viewJobBodyJobPostingTimestamp'}) else ""
        # job_qualification = job.find('div', {'data-testid': 'viewJobQualificationsContainer'})\
        #                         .find('ul')
        try:
            job_qualification = job.find('div', {'data-testid': 'viewJobQualificationsContainer'}).find('ul')
            if job_qualification:
                job_qualification = list(job_qualification.strings)
            else:
                job_qualification = []
        except AttributeError:
            job_qualification = []

        if job_qualification:
            job_qualification = list(job.find('div', {'data-testid': 'viewJobQualificationsContainer'})\
                                    .find('ul').strings)
            job_qualification = [x.strip() for x in job_qualification]
            # job_qualification = "\n".join(job_qualification)
        else:
            job_qualification = ""

        job_description = job.find('div', {'data-testid':'viewJobBodyJobFullDescriptionContent'})
        if job_description:
            job_description = list(job.find('div', {'data-testid':'viewJobBodyJobFullDescriptionContent'}).strings)
            job_description = [x.strip() for x in job_description]
            # job_description = "\n".join(job_description)
        else:
            job_description = ""

        job_posted_time = soup.find_all('span',{'data-testid':'viewJobBodyJobPostingTimestamp'})
        job_posted_time = job_posted_time[0].find('span',{'data-testid':'detailText'}).text.strip()
        # date_format = "%Y-%m-%d"
        date_format = "%Y-%m-%d"
        if "days" in  job_posted_time :
            job_posted_date = datetime.now() - timedelta(days=int(job_posted_time.split()[0]))
            job_posted_dates = job_posted_date.strftime(date_format)
        elif "hours" in job_posted_time:
            job_posted_date = datetime.now() - timedelta(hours=int(job_posted_time.split()[0]))
            job_posted_dates = job_posted_date.strftime(date_format)
        else:
            job_posted_dates = datetime.now()

        data = {
                'search_keyword_for_job':search_keyword_for_job,
                'job_title':job_title,
                'job_url':job_link,
                'company':company,
                'location': location, 
                'job_type': job_type, 
                'salary':salary, 
                'posted_on':job_posted_dates, 
                'job_qualification':job_qualification, 
                'job_description': job_description
                }
        data_list.append(data)
        # break
    return data_list
        

In [8]:
job_collection = ['Data Engineer','Frontend Developer', 'Backend Developer','Fullstack Developer', 'Business Analyst', 'DevOps Engineer', 'UI/UX', 'IT Manager']

# Create a list to store the DataFrames
df_list = []

# Navigate to URL
for item in job_collection:
    url = f'https://www.simplyhired.com/search?q={item}&l=united+states&t=1'
    next_page = url
    driver.get(url)
    # time.sleep(1)
    number_of_jobs = driver.find_element(By.XPATH, '/html/body/div[1]/div/main/div/div[1]/div[5]/p')
    number_of_pages = math.ceil(int(number_of_jobs.text)/int(20))
    print(f'Number of jobs for {item}:{number_of_jobs.text}   total_page:{number_of_pages}')

    i=1
    df = pd.DataFrame(columns=['search_keyword_for_job','job_title','job_url', 'company', 'location', 'job_type', 'salary', 'posted_on', 'job_qualification', 'job_description'])
    while next_page != None:
        print('Page Number:', i, ' Page Link: ', next_page)
        soup = scrape_page(next_page)
        page_data = scrape_one_page(next_page, soup,item)
        df = df.append(page_data)   
        df['company'] = df['company'].str.replace(',', '-')
        df['location'] = df['location'].str.replace(',', '-')
        df['salary'] = df['salary'].str.replace(',', '') 
        # i = i+1
        next_page = get_cursor(soup, i+1)
        time.sleep(1)
        i = i+1
         

    # Append the DataFrame to the list
    df_list.append(df)

df = pd.concat(df_list)
        
# Print page title
# print(driver.title)


Number of jobs for Data Engineer:104   total_page:6
Page Number: 1  Page Link:  https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=1
https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=1&job=dz7dm6ELpMc1Aha6xWQmV1gME8E_IVg1SJh4MguqgteJqTFiu88kYA
https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=1&job=0xk1JtjQJ6zifgmpdi8qvgbXV0GXlnBRQK8hYDJBtbzZLJy897Rarg
https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=1&job=71-kq0Y7Xhl9ifhN7p65yCkl2volsHBj-92KJ_vkj_1JvPdW1hhd7Q
https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=1&job=hdhvIFpFN9euvQxUJBX_8qH4l4MfWaEm925cmTBMVlMd9eQSkXv49A
https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=1&job=rnGWsMRFdyiSNsHmu0PvMebAGhhXgdhp3_aXucxUuSqdVzlkrdEZrg
https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=1&job=jUjeB31n7v3rYYuPszVg1vTLX-9e6iMfrw-qAouipRhUQhAVY9MuHg
https://www.simplyhired.com/search?q=Data Engineer&l=united+states&t=

/tmp/ipykernel_5743/3381847202.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


Page Number: 2  Page Link:  https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAQAUAAAAAAAAAAAAAAACDzC29AEBAQcSOzmSTQHnj1zgxtZXkawrecCyLIXAXFk%3D
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAQAUAAAAAAAAAAAAAAACDzC29AEBAQcSOzmSTQHnj1zgxtZXkawrecCyLIXAXFk%3D&job=ODGHSS10ONK8gEX9N6VdnF3yYnnNLcWuAUh_0gDiHU8Pl6q8FmgAXQ
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAQAUAAAAAAAAAAAAAAACDzC29AEBAQcSOzmSTQHnj1zgxtZXkawrecCyLIXAXFk%3D&job=EDEUJ49tJsQCDe1wbqPZynDmOJk6oKOUqILG6fy8Dzy36cZHCmxa7w
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAQAUAAAAAAAAAAAAAAACDzC29AEBAQcSOzmSTQHnj1zgxtZXkawrecCyLIXAXFk%3D&job=Q7uMTRTtSqLi5tcXKd3wNV3pXtlduoFWJU3vYolKr00luynjdzcBJA
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAQAUAAAAAAAAAAAAAAACDzC29AEBAQcSOzmSTQHnj1zgxtZXkawrecCyLIXAXFk%3D&job=5WChVghiZqlgNrAW9TN_c95zHZ5HxbUJz4cugAjTKzmFHs

/tmp/ipykernel_5743/3381847202.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


Page Number: 3  Page Link:  https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAgAoAAAAAAAAAAAAAAACDzC29AEBAQkE51A7uMgCOsIQAp5FP%2FE1ICOJocYmBamzQxX%2FmbqApQ%3D%3D
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAgAoAAAAAAAAAAAAAAACDzC29AEBAQkE51A7uMgCOsIQAp5FP%2FE1ICOJocYmBamzQxX%2FmbqApQ%3D%3D&job=YLAH4UOYYdkO8QN6FEtEpXTxmr1RAE6QpdvcYqQD43nlAWbgPsfsGw
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAgAoAAAAAAAAAAAAAAACDzC29AEBAQkE51A7uMgCOsIQAp5FP%2FE1ICOJocYmBamzQxX%2FmbqApQ%3D%3D&job=GoNbyhgf4aOi7agSix-vRpT-rUQTRnrw4qwLJsqw0AsOGCn4IYmg9g
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAgAoAAAAAAAAAAAAAAACDzC29AEBAQkE51A7uMgCOsIQAp5FP%2FE1ICOJocYmBamzQxX%2FmbqApQ%3D%3D&job=PqKOJbew1Y-x3IKgo58G-d4pfxttn-p8VpozIINjf4tGtFbw7RT0sA
https://www.simplyhired.com/search?q=Data+Engineer&l=united+states&t=1&cursor=ABQAAgAoAAAAAAAAAAAAAAACDzC29AEBAQkE51A7uMgCOsIQA

...

/tmp/ipykernel_5743/3381847202.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


Page Number: 6  Page Link:  https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABQBkAAAAAAAAAAAAAAACDzk3yQEBAQsDN8u9ydAlb%2FXEBPfsBkPQqqvvxcc4wqKzjiKPqFB6D7cHDTU%3D
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABQBkAAAAAAAAAAAAAAACDzk3yQEBAQsDN8u9ydAlb%2FXEBPfsBkPQqqvvxcc4wqKzjiKPqFB6D7cHDTU%3D&job=SosqcXWoW3xzUY235gm3TbxlBY7FM9hCQRNr5NiTZARSRDjOI6uFwA
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABQBkAAAAAAAAAAAAAAACDzk3yQEBAQsDN8u9ydAlb%2FXEBPfsBkPQqqvvxcc4wqKzjiKPqFB6D7cHDTU%3D&job=q14X70-UOfgYmr0Xb_aN27P3Lv1IjoTmR8jube1BODHMWVk4yxCpbQ
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABQBkAAAAAAAAAAAAAAACDzk3yQEBAQsDN8u9ydAlb%2FXEBPfsBkPQqqvvxcc4wqKzjiKPqFB6D7cHDTU%3D&job=_dAsE9A7XGxQipfs9bdzoxdARLUqymKUMUvtn98mO-PPjxx9Hkv09Q
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABQBkAAAAAAAAAAAAAAACDzk3yQEBAQsDN8u9ydAlb%2FXEBPfsBkPQqqvvxc

/tmp/ipykernel_5743/3381847202.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


Page Number: 7  Page Link:  https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABgB4AAAAAAAAAAAAAAACDzk3yQEBAQkPOOspKOR3iUc400x2iz8730dEWQBy0GCwTtKZ
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABgB4AAAAAAAAAAAAAAACDzk3yQEBAQkPOOspKOR3iUc400x2iz8730dEWQBy0GCwTtKZ&job=t9sOvLpptIUtwZD1-DJZQXTTDn6dp7HNpgn89Bct1yQmxCElGnLa0g
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABgB4AAAAAAAAAAAAAAACDzk3yQEBAQkPOOspKOR3iUc400x2iz8730dEWQBy0GCwTtKZ&job=4SQigNrTOSa0O4NvxlnYwk2-4SBmLie4x8P-T9nsjJNrGvMmTV3WvQ
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABgB4AAAAAAAAAAAAAAACDzk3yQEBAQkPOOspKOR3iUc400x2iz8730dEWQBy0GCwTtKZ&job=l3r3qdVMR4Rr-BbK6gJ-3FgALujwKop4b-etK03uRBfOSbrJT44Fjw
https://www.simplyhired.com/search?q=IT+Manager&l=united+states&t=1&cursor=ABQABgB4AAAAAAAAAAAAAAACDzk3yQEBAQkPOOspKOR3iUc400x2iz8730dEWQBy0GCwTtKZ&job=l4zvwBSsvHQVzmOmW9s4x-_nOlUrAEcojNDnThDm4pkAI9k0426

/tmp/ipykernel_5743/3381847202.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(page_data)


In [9]:
df.to_csv('simply_hired_test102.csv', index=False)

In [10]:
# Quit the driver
driver.quit()

In [ ]:
import requests
basescrape = requests.get('https://api.crawlbase.com/?token=tcGRf4X6PbGB1EzaLGxCyg&url=https://www.simplyhired.com/search?q=data+engineerr&l=united+states&t=1')

In [ ]:
basescrape.status_code